# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [200]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [201]:
data = pd.read_csv('customer_product_sales.csv')

In [202]:
data.head(20)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
5,83106,Hope,Bowers,6484922,229,Bread - Hot Dog Buns,22
6,11253,Nicolas,Holmes,4878478,229,Bread - Hot Dog Buns,3
7,35107,Judith,Miranda,2353899,229,Bread - Hot Dog Buns,9
8,15088,Ruby,Lowe,37021,229,Bread - Hot Dog Buns,4
9,26031,Josephine,Curry,2327232,229,Bread - Hot Dog Buns,7


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [203]:
mydataframe = data.groupby(["CustomerID", "ProductName"]).agg({'Quantity':'sum'})
mydataframe.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [204]:
#filtered = data[(data["CustomerID"] == 97753) & (data["ProductName"] == "Anchovy Paste - 56 G Tube")]
#filtered

In [205]:
mydataframe2 = pd.pivot_table(mydataframe, values = "Quantity", index = "ProductName", columns = "CustomerID", 
                              fill_value = 0)
mydataframe2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [206]:
from scipy.spatial.distance import pdist, squareform

sim_matrix = squareform(pdist(mydataframe2.T, "euclidean"))

sim_matrix

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [207]:
customerIDs = mydataframe2.columns.values

In [208]:
similaritydf = pd.DataFrame(1/(1 + squareform(pdist(mydataframe2.T, "euclidean"))), 
                         index = customerIDs, columns = customerIDs)

display(similaritydf.head())

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [209]:
similarities = pd.DataFrame(similaritydf[33].sort_values(ascending=False)[:5])
display(similarities)
print(similarities.index)

,33
33,1.000000
264,0.087047
3535,0.087047
3317,0.087047
2503,0.085983


Int64Index([33, 264, 3535, 3317, 2503], dtype='int64')


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [210]:
sim_custs = data[data["CustomerID"].isin(similarities.index)]
sim_custs.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
159,264,Virgil,Small,24688,270,Cheese Cloth No 100,1
242,3535,Lana,Peters,2399506,270,Cheese Cloth No 100,1
527,33,Lindsay,Santana,1899667,432,Beer - Original Organic Lager,1
1017,33,Lindsay,Santana,6509275,417,"Lamb - Pieces, Diced",1
1038,2503,Kyle,Reeves,182355,417,"Lamb - Pieces, Diced",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [212]:
sim_cust2 = pd.DataFrame(sim_custs.groupby("ProductName").agg({'Quantity':'sum'}).sort_values('Quantity', ascending = False)).reset_index()
sim_cust2.head()


,ProductName,Quantity
0,"Pepper - Black, Whole",4
1,Butter - Unsalted,3
2,Grouper - Fresh,3
3,"Veal - Inside, Choice",3
4,Veal - Sweetbread,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [157]:
mydataframe2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In [162]:
m = sim_cust2.merge(mydataframe2, left_on = "ProductName", right_on = "ProductName")
m2 = m[["ProductName", "Quantity", 33]]
m2.head()

,ProductName,Quantity,33
0,"Pepper - Black, Whole",4,1
1,Butter - Unsalted,3,0
2,Grouper - Fresh,3,2
3,"Veal - Inside, Choice",3,2
4,Veal - Sweetbread,3,1


In [167]:
sim_cust2_no33 = m2[m2[33] == 0]
sim_cust2_no33.head()

top5 = m2["ProductName"][:5]
top5

0    Pepper - Black, Whole
1        Butter - Unsalted
2          Grouper - Fresh
3    Veal - Inside, Choice
4        Veal - Sweetbread
Name: ProductName, dtype: object

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [176]:
recommendations = {}

customers = list(set(data["CustomerID"]))

# Encuentro los 5 usuarios similares:
similarities = pd.DataFrame(similaritydf[33].sort_values(ascending=False)[:5])

# Creo el dataframe con los registros de esos usuarios:
sim_custs = data[data["CustomerID"].isin(similarities.index)]

# Agrego los registros por 
sim_cust2 = pd.DataFrame(sim_custs.groupby("ProductName").agg({'Quantity':'sum'}).sort_values('Quantity', ascending = False)).reset_index()


[83973, 59399, 92168, 49159, 18441, 22536, 86028, 75791, 96272, 32785, 47122, 75793, 51224, 40985, 28702, 33, 6185, 18480, 86065, 94258, 67636, 81981, 90174, 10311, 28751, 51279, 90193, 30803, 55380, 28756, 55381, 41045, 12381, 30815, 92255, 86111, 67682, 16481, 88159, 65650, 57459, 18552, 86136, 8316, 79997, 77950, 82057, 2187, 65678, 86167, 57497, 18591, 61601, 71844, 4261, 14501, 20648, 90284, 6317, 8367, 22705, 10418, 32946, 8372, 88245, 41147, 90300, 18622, 28867, 65731, 39110, 47303, 200, 47305, 28875, 6352, 45266, 94420, 35028, 35034, 65757, 6367, 22751, 78048, 82146, 26850, 78054, 16614, 94438, 30954, 90347, 59626, 12529, 82161, 24820, 18680, 65785, 51451, 18683, 53501, 63742, 88318, 18688, 45313, 92418, 78082, 57606, 43270, 264, 16650, 96522, 96524, 71948, 12561, 18706, 69906, 78104, 2329, 16664, 51483, 55580, 63771, 33051, 18720, 35107, 65830, 80167, 18728, 69931, 49454, 6447, 28976, 96560, 12596, 78133, 57654, 61750, 28984, 80184, 12598, 84280, 63803, 59703, 59710, 59711, 49

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.